In [1]:
import sympy as sp
from sympy import sin,cos
import numpy as np
from sympy import diff
from IPython.display import display

In [2]:
from commons import SurfaceCurve,Surface,MethodOfOrthonormalFrames,CovariantDeriv

In [3]:
u,v = sp.symbols("u,v")
du,dv=sp.symbols("du,dv")

<pre>First we shall start with the case of a surface p(u, v) in the space. Let a tangent vector
X(t) be given at each point on curve p(t) = p(u(t), v(t)) on this surface. (We call
this a vector field along the curve.) Since X(t) is a vector in the space, it has three
components and by differentiating every component for t, we define X`(t) (or dX/dt ).
X`(t) also has three components, so we can assume that it is also a vector in the
space, but it is not necessarily tangent to the surface p(u, v). We write X`(t) as a sum
of a tangent vector and normal vector of this surface</pre>

![](./images/im3.4.1.png)

In [4]:
class CovariantDeriv:
    def __init__(self,surface_curve:SurfaceCurve):
        self.surface_curve=surface_curve
        self.curve=self.surface_curve.curve_param
        self.u=self.surface_curve.u
        self.v=self.surface_curve.v
        #Any parameterization. Not necesarilly by distance.
        self.t=self.surface_curve.s
        self.surface=self.surface_curve.surface
        self.ort=MethodOfOrthonormalFrames(self.surface)
        self.W_u,self.W_v=self.ort.calc_mat_W()
        self.e1=sp.Array(self.ort.e1)
        self.e2=sp.Array(self.ort.e2)
        self.e3=sp.Array(self.ort.e3)
        self.e1,self.e2,self.e3=[(x/sp.sqrt(np.dot(x,x))).simplify() for x in [self.e1,self.e2,self.e3]]
        
    def covariant_deriv(self,X):
        #X = X[0]*surface.e1 + X[1]*surface.e2
        
        #Covariant deriv <=> the tangent vector
        curve=self.curve
        t=self.t
        xi1,xi2=X
        w12u,w21u=self.W_u[0,1],self.W_u[1,0]
        w12v,w21v=self.W_v[0,1],self.W_v[1,0]
        u_t,v_t=self.surface_curve.curve_param
        e1,e2=sp.Array(self.e1),sp.Array(self.e2)
        w12u,w21u,w12v,w21v,e1,e2=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in [w12u,w21u,w12v,w21v,e1,e2]]
        dxi1dt=diff(xi1,t)
        dxi2dt=diff(xi2,t)
        #u12/dt = (a*du+b*dv)/dt = a*u' + b*v'
        return ((dxi1dt+xi2*(w21u*diff(u_t,t)+w21v*diff(v_t,t)))*e1 + \
                (dxi2dt+xi1*(w12u*diff(u_t,t)+w12v*diff(v_t,t)))*e2).simplify()

    def tangent_vector(self,X):
        return self.covariant_deriv(X)
    
    def normal_vector(self,X):
        curve=self.curve
        #A_X
        t=self.t
        xi1,xi2=X
        w13u=self.W_u[0,2]
        w13v=self.W_v[0,2]
        u_t,v_t=self.surface_curve.curve_param
        e3=self.e3
        w13u,w13v,e3=[x.subs(self.u,u_t).subs(self.v,v_t).simplify() for x in [w13u,w13v,e3]]
        #u12/dt = (a*du+b*dv)/dt = a*u' + b*v'
        return ((xi1*(w13u*diff(u_t,t)+w13v*diff(v_t,t))+ \
                xi2*(w13u*diff(u_t,t)+w13v*diff(v_t,t)))*e3).simplify()
    
    def X_deriv(self,X):
        return self.tangent_vector(X)+self.normal_vector(X)
    

In [5]:
t=sp.symbols('t')
surf=Surface(u,v,[
    u,
    v,
    u**2+v**2
])
cur=[
    t, #u(t)
    -t # v(t)
]
sc=SurfaceCurve(u,v,t,surf,cur)
cov = CovariantDeriv(sc)

In [6]:
X=[
    u-v,v**2,#X = (u-v)e1 + (v**2)e2
]
print("DX/dt:")
display(cov.covariant_deriv(X))
print("A_X:")
display(cov.normal_vector(X))

DX/dt:


[4*t*(-4*t**2*u + 4*t**2*v + v**2*sqrt((32*t**4 + 12*t**2 + 1)/(16*t**4 + 8*t**2 + 1))*sqrt(4*t**2 + 1))/(sqrt(4*t**2 + 1)*(32*t**4 + 12*t**2 + 1)), 4*t*sqrt(4*t**2 + 1)*(-u + v)/(32*t**4 + 12*t**2 + 1), 8*t**2*(u + v**2*sqrt((32*t**4 + 12*t**2 + 1)/(16*t**4 + 8*t**2 + 1))*sqrt(4*t**2 + 1) - v)/(sqrt(4*t**2 + 1)*(32*t**4 + 12*t**2 + 1))]

A_X:


[4*t*(-u - v**2 + v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1)), 4*t*(u + v**2 - v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1)), 2*(u + v**2 - v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1))]

In [110]:
print("X'")
cov.X_deriv(X)

X'


[4*t*(-4*t**2*u + 4*t**2*v + v**2*sqrt((32*t**4 + 12*t**2 + 1)/(16*t**4 + 8*t**2 + 1))*sqrt(4*t**2 + 1))/(sqrt(4*t**2 + 1)*(32*t**4 + 12*t**2 + 1)) + 4*t*(-u - v**2 + v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1)), 4*t*sqrt(4*t**2 + 1)*(-u + v)/(32*t**4 + 12*t**2 + 1) + 4*t*(u + v**2 - v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1)), 8*t**2*(u + v**2*sqrt((32*t**4 + 12*t**2 + 1)/(16*t**4 + 8*t**2 + 1))*sqrt(4*t**2 + 1) - v)/(sqrt(4*t**2 + 1)*(32*t**4 + 12*t**2 + 1)) + 2*(u + v**2 - v)/(sqrt(4*t**2 + 1)*(8*t**2 + 1))]

In [111]:
#Matrix W
print("W = (w_ij)")
cov.W_u*du+cov.W_v*dv

W = (w_ij)


Matrix([
[                                                                                                                           0, 4*du*v*sqrt(4*u**2 + 1)/(sqrt((16*u**4 + 16*u**2*v**2 + 8*u**2 + 4*v**2 + 1)/(16*u**4 + 8*u**2 + 1))*(16*u**4 + 8*u**2 + 1)),  2*du/(sqrt((16*u**4 + 16*u**2*v**2 + 8*u**2 + 4*v**2 + 1)/(16*u**4 + 8*u**2 + 1))*(4*u**2 + 1))],
[-4*du*v*sqrt((16*u**4 + 16*u**2*v**2 + 8*u**2 + 4*v**2 + 1)/(16*u**4 + 8*u**2 + 1))/(sqrt(4*u**2 + 1)*(4*u**2 + 4*v**2 + 1)),                                                                                                                            0, -8*du*u*v/(sqrt(4*u**2 + 1)*(4*u**2 + 4*v**2 + 1)) + 2*dv*sqrt(4*u**2 + 1)/(4*u**2 + 4*v**2 + 1)],
[                     -2*du*sqrt((16*u**4 + 16*u**2*v**2 + 8*u**2 + 4*v**2 + 1)/(16*u**4 + 8*u**2 + 1))/(4*u**2 + 4*v**2 + 1),       8*du*u*v*sqrt(4*u**2 + 1)/(16*u**4 + 16*u**2*v**2 + 8*u**2 + 4*v**2 + 1) - 2*dv*sqrt(4*u**2 + 1)/(4*u**2 + 4*v**2 + 1),                                   